# End-to-End Demonstration: Google Classroom

*Goal*: run the Google Classroom Extractor and then upload the results into an `LMS` database.

## Software Requirements

1. Be sure to install Python 3.9; if you have multiple versions, make sure that the `python` command runs version 3.9.x. You can confirm your version by running `python --version` at a command prompt.
1. Microsoft SQL Server 2017 or 2019, in Windows or Linux.

## Getting Started

1. Confirm you have [poetry](https://python-poetry.org) installed (`poetry --version`).
1. Follow the [Google Classroom setup instructions](../google-classroom/README.md) in order to create a `service-account.json` file.
1. Follow the [notebook instructions](README.md) to install dependencies used by this notebook.
1. Create an `LMS` database in SQL Server.

In [1]:
# Load some utilities
from IPython.display import display, Markdown

# Setup logging
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Prepare Input Data

Update the variables in the next block as needed.

In [2]:
CLASSROOM_ACCOUNT = "admin@ibamonitoring.org"
START_DATE = "2020-08-17"
END_DATE = "2021-05-23"
LOG_LEVEL = "INFO"
OUTPUT_DIRECTORY = "gc-data"
SYNC_DATABASE_DIRECTORY=OUTPUT_DIRECTORY
DB_ENGINE = "mssql"
DB_SERVER = "localhost"
DB_NAME = "LMS"
DB_PORT = 1433
EXTRACT_ASSIGNMENTS = True
EXTRACT_ACTIVITIES = False
EXTRACT_ATTENDANCE = False
EXTRACT_GRADES = False

## Run the Google Classroom Extractor

In [3]:
from edfi_google_classroom_extractor.helpers.arg_parser import MainArguments as gc_args
from edfi_google_classroom_extractor import facade

arguments = gc_args(
    classroom_account=CLASSROOM_ACCOUNT,
    log_level=LOG_LEVEL,
    output_directory=OUTPUT_DIRECTORY,
    usage_start_date=START_DATE,
    usage_end_date=END_DATE,
    sync_database_directory=SYNC_DATABASE_DIRECTORY,
    extract_assignments=EXTRACT_ASSIGNMENTS,
    extract_activities=EXTRACT_ACTIVITIES,
    extract_attendance=EXTRACT_ATTENDANCE,
    extract_grades=EXTRACT_GRADES,
)

facade.run(arguments)

INFO:edfi_google_classroom_extractor.facade:Starting Ed-Fi LMS Google Classroom Extractor
INFO:edfi_google_classroom_extractor.api.courses:Pulling course data
INFO:edfi_google_classroom_extractor.facade:Writing LMS UDM Sections to CSV file
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => gc-data\sections\2021-09-03-16-39-48.csv
INFO:edfi_google_classroom_extractor.api.students:Pulling student data
INFO:edfi_google_classroom_extractor.api.teachers:Pulling teacher data
INFO:edfi_google_classroom_extractor.facade:Writing LMS UDM Users to CSV file
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => gc-data\users\2021-09-03-16-39-48.csv
INFO:edfi_google_classroom_extractor.facade:Writing LMS UDM UserSectionAssociations to CSV files
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => gc-data\section=126946994497\section-associations\2021-09-03-16-39-48.csv
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => gc-data\section=127236

## Run the Learning Management System Data Store Loader (LMS-DS-Loader)

The default setup below uses Windows integrated security. For username/password security, please review the commented-out code.

In [5]:
from edfi_lms_ds_loader.helpers.argparser import MainArguments as lms_args
from edfi_lms_ds_loader import loader_facade

arguments = lms_args(
    OUTPUT_DIRECTORY,
    DB_ENGINE,
    LOG_LEVEL,
    DB_SERVER,
    DB_NAME,
    DB_PORT,
)
arguments.build_mssql_adapter_with_integrated_security()

# For password auth, comment out the lines above and uncomment the next ones:
# arguments = lms_args(
#     OUTPUT_DIRECTORY,
#     DB_ENGINE,
#     LOG_LEVEL,
#     DB_SERVER,
#     DB_PORT,
#     DB_NAME,
# )
# arguments.build_mssql_adapter(your_db_user, your_db_password)

loader_facade.run_loader(arguments)

INFO:edfi_lms_ds_loader.loader_facade:Begin loading files into the LMS Data Store (DS)...
INFO:edfi_lms_ds_loader.migrator:Begin database auto-migration...
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_sql_adapter.sql_adapter:Updating migration journal table
INFO:edfi_lms_ds_loader.migrator:Done wi